In [1]:
from IPython.display import display, HTML

display(HTML('''<script>
    function ClickConnect(){
        console.log("clicked on connect button");
        document.querySelector("colab-connect-button").click();
    }
    setInterval(ClickConnect, 60000);
</script>'''))

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, top_k_accuracy_score
import seaborn as sns
import cv2

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, VGG19, ResNet50
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow as tf


In [3]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [4]:
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_dataset = datasets.OxfordIIITPet(root='data', split='trainval',
                                       target_types='category', transform=transform, download=True)
test_dataset = datasets.OxfordIIITPet(root='data', split='test',
                                      target_types='category', transform=transform, download=True)

from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

num_classes = len(set(train_dataset._labels))  # 37

100%|██████████| 792M/792M [00:34<00:00, 22.8MB/s]
100%|██████████| 19.2M/19.2M [00:01<00:00, 10.5MB/s]


In [5]:
base_dir = "data/oxford-iiit-pet/images"
image_files = [f for f in os.listdir(base_dir) if f.lower().endswith('.jpg')]
labels = [f.split('_')[0].lower() for f in image_files]

df = pd.DataFrame({
    'filename': image_files,
    'label': labels
})

class_names = sorted(df['label'].unique())

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

In [6]:
IMG_SIZE = 224
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=vgg_preprocess
)

val_datagen = ImageDataGenerator(preprocessing_function=vgg_preprocess)
test_datagen = ImageDataGenerator(preprocessing_function=vgg_preprocess)

train_generator = train_datagen.flow_from_dataframe(
    train_df, directory=base_dir, x_col='filename', y_col='label',
    target_size=(IMG_SIZE, IMG_SIZE), class_mode='sparse', batch_size=BATCH_SIZE
)

val_generator = val_datagen.flow_from_dataframe(
    val_df, directory=base_dir, x_col='filename', y_col='label',
    target_size=(IMG_SIZE, IMG_SIZE), class_mode='sparse', batch_size=BATCH_SIZE, shuffle=False
)

test_generator = test_datagen.flow_from_dataframe(
    test_df, directory=base_dir, x_col='filename', y_col='label',
    target_size=(IMG_SIZE, IMG_SIZE), class_mode='sparse', batch_size=BATCH_SIZE, shuffle=False
)

Found 5320 validated image filenames belonging to 35 classes.
Found 592 validated image filenames belonging to 35 classes.
Found 1478 validated image filenames belonging to 35 classes.


In [7]:
from tensorflow.keras.applications import VGG16, VGG19, ResNet50
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess

from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

def build_model(base_model_fn, input_shape=(224, 224, 3), num_classes=37, preprocess_fn=None, train_base=False):
    base_input = Input(shape=input_shape)
    x = preprocess_fn(base_input) if preprocess_fn else base_input

    base_model = base_model_fn(weights='imagenet', include_top=False, input_tensor=x)
    base_model.trainable = train_base

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_input, outputs=output)
    model.compile(optimizer=Adam(1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

In [8]:
def train_model(model, train_gen, val_gen, model_name, epochs=10):
    checkpoint = ModelCheckpoint(f'{model_name}.keras', save_best_only=True, monitor='val_accuracy', mode='max')
    earlystop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=epochs,
        callbacks=[checkpoint, earlystop],
        verbose=1
    )

    return model, history

In [9]:
num_classes = len(class_names)

vgg16_model = build_model(VGG16, num_classes=num_classes, preprocess_fn=vgg_preprocess)
vgg16_model, vgg16_history = train_model(vgg16_model, train_generator, val_generator, 'vgg16_model')

test_loss, test_acc = vgg16_model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 109s 551ms/step - accuracy: 0.0338 - loss: 23.0918 - val_accuracy: 0.0642 - val_loss: 14.9831
Epoch 2/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 80s 477ms/step - accuracy: 0.0530 - loss: 17.5533 - val_accuracy: 0.1115 - val_loss: 11.6537
Epoch 3/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 81s 482ms/step - accuracy: 0.0876 - loss: 14.5253 - val_accuracy: 0.1622 - val_loss: 9.1702
Epoch 4/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 81s 483ms/step - accuracy: 0.1315 - loss: 12.4413 - val_accuracy: 0.2331 - val_loss: 7.4562
Epoch 5/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 81s 483ms/step - accuracy: 0.1628 - loss: 10.6898 - val_accuracy: 0.2905 - val_loss: 6.2354
Epoch 6/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 80s 480ms/step - accuracy: 0.2023 - loss: 9.5240 - val_accuracy: 0.3429 - val_loss: 5.2805
Epoch 7/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 81s 484ms/step - accuracy: 0.2317 - loss: 8.4966 - val_accuracy: 0.4054 - val_loss: 4.6112
Epoch 8/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 80s 480ms/step - accuracy: 0.2497 -

In [ ]:
vgg19_model = build_model(VGG19, num_classes=num_classes, preprocess_fn=vgg_preprocess)
vgg19_model, vgg19_history = train_model(vgg19_model, train_generator, val_generator, 'vgg19_model')

test_loss, test_acc = vgg19_model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
 82/167 ━━━━━━━━━━━━━━━━━━━━ 42s 500ms/step - accuracy: 0.0358 - loss: 23.4882

In [ ]:
resnet_train_gen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=resnet_preprocess
)

resnet_val_gen = ImageDataGenerator(preprocessing_function=resnet_preprocess)

resnet_train_generator = resnet_train_gen.flow_from_dataframe(
    train_df, directory=base_dir, x_col='filename', y_col='label',
    target_size=(IMG_SIZE, IMG_SIZE), class_mode='sparse', batch_size=BATCH_SIZE
)

resnet_val_generator = resnet_val_gen.flow_from_dataframe(
    val_df, directory=base_dir, x_col='filename', y_col='label',
    target_size=(IMG_SIZE, IMG_SIZE), class_mode='sparse', batch_size=BATCH_SIZE, shuffle=False
)

resnet_model = build_model(ResNet50, num_classes=num_classes, preprocess_fn=resnet_preprocess)
resnet_model, resnet_history = train_model(resnet_model, resnet_train_generator, resnet_val_generator, 'resnet_model')

test_loss, test_acc = resnet_model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

y_true = test_generator.classes
y_pred = vgg16_model.predict(test_generator)
y_pred = np.argmax(y_pred, axis=1)

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=train_generator.class_indices.keys(), yticklabels=train_generator.class_indices.keys())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import matplotlib.pyplot as plt

cat_classes = [c.replace(' ', '_').lower() for c in [
    'Abyssinian', 'Bengal', 'Birman', 'Bombay', 'British Shorthair',
    'Egyptian Mau', 'Maine Coon', 'Persian', 'Ragdoll',
    'Russian Blue', 'Siamese', 'Sphynx'
]]

dog_classes = [d.replace(' ', '_').lower() for d in [
    'American Bulldog', 'American Pit Bull Terrier', 'Basset Hound', 'Beagle',
    'Boxer', 'Chihuahua', 'English Cocker Spaniel', 'English Setter',
    'German Shorthaired Pointer', 'Great Pyrenees', 'Havanese', 'Japanese Chin',
    'Keeshond', 'Leonberger', 'Miniature Pinscher', 'Newfoundland',
    'Pomeranian', 'Pug', 'Saint Bernard', 'Samoyed', 'Scottish Terrier',
    'Shiba Inu', 'Staffordshire Bull Terrier', 'Wheaten Terrier', 'Yorkshire Terrier'
]]

image_dir = 'data/oxford-iiit-pet/images'
image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]

data = []
for file in image_files:
    breed = file.rsplit('_', 1)[0].lower()
    if breed in cat_classes:
        label = 'cat'
    elif breed in dog_classes:
        label = 'dog'
    else:
        continue
    data.append({'filename': file, 'label': label})

df = pd.DataFrame(data)

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_dataframe(
    train_df,
    directory=image_dir,
    x_col='filename',
    y_col='label',
    target_size=(224, 224),
    class_mode='binary',
    batch_size=32,
    shuffle=True
)

val_gen = datagen.flow_from_dataframe(
    val_df,
    directory=image_dir,
    x_col='filename',
    y_col='label',
    target_size=(224, 224),
    class_mode='binary',
    batch_size=32,
    shuffle=False
)

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

binary_model = Model(inputs=base_model.input, outputs=output)
binary_model.compile(optimizer=Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])

binary_model.fit(train_gen, validation_data=val_gen, epochs=5)

loss, accuracy = binary_model.evaluate(val_gen)
print(f"Validation Accuracy: {accuracy:.4f}")

val_gen.reset()
y_pred_probs = binary_model.predict(val_gen)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()
y_true = val_gen.classes

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Cat', 'Dog'])
disp.plot()
plt.show()


In [ ]:
from sklearn.metrics import top_k_accuracy_score


y_true = test_generator.classes
y_pred = vgg16_model.predict(test_generator)

top_3_acc = top_k_accuracy_score(y_true, y_pred, k=3, labels=np.arange(num_classes))
top_5_acc = top_k_accuracy_score(y_true, y_pred, k=5, labels=np.arange(num_classes))

print(f"Top-3 Accuracy: {top_3_acc}")
print(f"Top-5 Accuracy: {top_5_acc}")

In [ ]:
y_pred = vgg19_model.predict(test_generator)

top_3_acc = top_k_accuracy_score(y_true, y_pred, k=3, labels=np.arange(num_classes))
top_5_acc = top_k_accuracy_score(y_true, y_pred, k=5, labels=np.arange(num_classes))

print(f"Top-3 Accuracy: {top_3_acc}")
print(f"Top-5 Accuracy: {top_5_acc}")

In [ ]:
y_pred = resnet_model.predict(test_generator)

top_3_acc = top_k_accuracy_score(y_true, y_pred, k=3, labels=np.arange(num_classes))
top_5_acc = top_k_accuracy_score(y_true, y_pred, k=5, labels=np.arange(num_classes))

print(f"Top-3 Accuracy: {top_3_acc}")
print(f"Top-5 Accuracy: {top_5_acc}")

Grand Cam